# IMPORT LIBRARIES

In [1]:
import pandas as pd
import numpy as np

# Data Preprocess

## Helper Functions

In [2]:
# dtype is consistent over all observations
def consistent(df, col):
    '''
        gets a list and returns the first datatype
    '''
    for i in range(len(df)):
        if  i == 0:
            first_dtype = type(df.loc[i, col])
        else:
            if first_dtype != type(df.loc[i, col]): # case when not consistent
                return False
    return True

def checktype_num(col_name, u_values):
    '''
        gets a list and returns the first datatype
    '''
    print(col_name)
    for val in u_values:
        if (isinstance(val, np.int64)) or (isinstance(val, np.float64)) or (isinstance(val, int)) or (isinstance(val, float)):
            return True
    return False



def checktype_str(col_name, u_values):
    '''
        gets a list and returns the first datatype
    '''
    for val in u_values:
        if (isinstance(val, str)):
            return True
    return False

def col_values_info(df, col):
    '''
        function to find the unique values in a pandas df[col]:
        unique values in col and there info
    '''
    shape = df.shape
    unique_values = list(df[col].unique())
    na_val = df[col].isna().sum()
    len_list = len(unique_values)
#     occr_val = [len(df[df[col] == j]) for j in unique_values]
    is_consistent = consistent(df, col)
    data_type = type(df[col].iloc[0])
    values_counts = dict(df[col].value_counts(dropna=False))
    
    info = dict(col_name=col, values_counts=values_counts, num_na=na_val,
                con_dtype = is_consistent, unq_col = (shape[0] == len_list))
    return info

def get_transform_list(df, info):
    '''
        returns columns that are continuous, categorical, categorical but not consistent dtype, numerical, numerical but not consistent dtype
    '''
    high_card = []
#     [key for key, data in info.items() if data['unq_col']]
    
    numeric = [col for col in df.select_dtypes(include=['number']) if col not in high_card + ['saleprice'] ]
    numeric_issue = [col for col in numeric if df[col].isna().sum() != 0]
    numeric = [col for col in numeric if col not in numeric_issue]


    categorical = [col for col in df.select_dtypes(exclude=['number'])]
    categorical_issue = [col for col in categorical if df[col].isna().sum() != 0]
    categorical = [col for col in categorical if col not in categorical_issue]
    
    # print(len(numeric_issue) + len(numeric) + len(categorical_issue) + len(categorical) + len(high_card))
    return high_card, numeric, numeric_issue, categorical, categorical_issue

def rows_to_change(df, col_list):
    '''
        Identifies:
            col with small number of missing rows < 20%
            col with large number of missing rows > 50%
    '''
    train_shape = df.shape
    small_m_rows = []
    large_m_rows = []
    other_m_rows = []
    for col in col_list:
        # print(col)
        if df[col].isna().sum() < (train_shape[0]*0.2): # col is missing less then 20% of values
            small_m_rows.append(col)
        elif df[col].isna().sum()  > (train_shape[0]*0.5): # col is missing more then 50% of values
            large_m_rows.append(col)
        else: # col is missing between 20% and 50% 
            other_m_rows.append(col)   
    return small_m_rows, large_m_rows, other_m_rows

def remove_outliers(df, columns, low=0.01, high=0.99):
    '''
        Removes outlier that are 1% from the left end data and 1% of the right end data
    '''
    for col in columns:
        q_low = df[col].quantile(low)
        q_hi  = df[col].quantile(high)
        temp = df[(df[col] < q_hi) & (df[col] > q_low)]
    return df

## Import Data

In [3]:
train = pd.read_csv('../../datasets/train.csv')
test = pd.read_csv('../../datasets/test.csv')

## Change Column Names to Lowercase

In [4]:
train.columns = train.columns.str.lower().str.replace(' ', '_')
test.columns = test.columns.str.lower().str.replace(' ', '_')

## Get Columns with issues

In [5]:
train_col_info = {i:col_values_info(train, str(i)) for i in train.columns}
high_card, numeric, numeric_issue, categorical, categorical_issue = get_transform_list(train, train_col_info)

test_col_info = {i:col_values_info(test, str(i)) for i in test.columns}
high_card_test, numeric_test, numeric_issue_test_test, categorical_test, categorical_issue_test = get_transform_list(test, test_col_info)

train_copy = train.copy()

categorical_columns = categorical + categorical_issue
numeric_columns = numeric + numeric_issue

categorical_columns_test = categorical_test + categorical_issue_test
numeric_columns_test = numeric_test + numeric_issue_test_test

## Categorical Feature Data Preprocess

In [6]:
cat_small_m, cat_large_m, cat_other_m = rows_to_change(train, categorical_issue)

### Fill missing Values

In [7]:
train_copy[cat_small_m] = train_copy[cat_small_m].fillna('None')
test[cat_small_m] = test[cat_small_m].fillna('None')

train_copy[cat_large_m] = train_copy[cat_large_m].fillna('None') 
test[cat_large_m] = test[cat_large_m].fillna('None')

train_copy[cat_other_m] = train_copy[cat_other_m].fillna('None')
test[cat_other_m] = test[cat_other_m].fillna('None')

In [8]:
train_copy[categorical + categorical_issue] = train_copy[categorical + categorical_issue].astype('category')

test[categorical + categorical_issue] = test[categorical + categorical_issue].astype('category')

- **In terms of Filling in missing values for Categorical if houses dont have specific fetures then the values should be None**

## Numeric Feature Data Preprocess

In [9]:
num_small_m, num_large_m, num_other_m = rows_to_change(train, numeric_issue)

### Fill missing Values

In [10]:
train_copy[num_small_m] = train_copy[num_small_m].fillna(0)
test[num_small_m] = test[num_small_m].fillna(0)

### Change Data Type

In [11]:
train_copy[numeric + numeric_issue] = train_copy[numeric + numeric_issue].astype(float)
test[numeric + numeric_issue] = test[numeric + numeric_issue].astype(float)

## Get High Correlated Columns with y

In [12]:
def high_saleprice_corr_columns(df):
    return [col for col, val in df.corr()['saleprice'].abs().iteritems() if val > 0.4 and col != 'saleprice']
            
columns = high_saleprice_corr_columns(train_copy)

## Remove Outliers

In [13]:
train_copy = remove_outliers(train_copy, numeric + numeric_issue)
train_copy.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'g

## Make Additional Features

In [14]:
train_copy['actual_totrms_abvgrd'] = train_copy['totrms_abvgrd'] + train_copy['full_bath']
train_copy['g_gr_area'] = train_copy['garage_area'] + train_copy['gr_liv_area']
train_copy['total_sf'] = train_copy['1st_flr_sf'] + train_copy['total_bsmt_sf']

test['actual_totrms_abvgrd'] = test['totrms_abvgrd'] + test['full_bath']
test['g_gr_area'] = test['garage_area'] + test['gr_liv_area']
test['total_sf'] = test['1st_flr_sf'] + test['total_bsmt_sf']

## Write Cleaned Data to CSV

In [15]:
train_copy.to_csv('../../datasets/cleaned_data/train_preproc.csv')
test.to_csv('../../datasets/cleaned_data/test_preproc.csv')